## Matrix and Vocabulary Construction

# Código feito pelos monitores.

In [ ]:
import pandas as pd

import numpy as np

from scipy import sparse

import nltk
from nltk import bigrams
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer

import math
__author__ = "Thierry Barros"

indice_invertido = {}
dados_TF = {}



In [ ]:
news = pd.read_csv("../data/estadao_noticias_eleicao.csv", encoding="utf-8")

In [ ]:
content = news.titulo + " " + news.subTitulo + " " + news.conteudo
content = content.fillna("")

In [ ]:
def co_occurrence_matrix(corpus):
    vocab = set(corpus)
    vocab = list(vocab)
    n = len(vocab)
   
    vocab_to_index = {word:i for i, word in enumerate(vocab)}
    
    bi_grams = list(bigrams(corpus))

    bigram_freq = nltk.FreqDist(bi_grams).most_common(len(bi_grams))

    I=list()
    J=list()
    V=list()
    
    for bigram in bigram_freq:
        current = bigram[0][1]
        previous = bigram[0][0]
        count = bigram[1]

        I.append(vocab_to_index[previous])
        J.append(vocab_to_index[current])
        V.append(count)
        
    co_occurrence_matrix = sparse.coo_matrix((V,(I,J)), shape=(n,n))

    return co_occurrence_matrix, vocab_to_index

#### Removing punctuation

In [ ]:
tokenizer = RegexpTokenizer(r'\w+')
tokens_lists = content.apply(lambda text: tokenizer.tokenize(text.lower()))

#### Removing stopwords

In [ ]:
stopword_ = stopwords.words('portuguese')
filtered_tokens = tokens_lists.apply(lambda tokens: [token for token in tokens if token not in stopword_])

#### Transforming list of lists into one list

In [ ]:
tokens = [token for tokens_list in filtered_tokens for token in tokens_list]

In [ ]:
matrix, vocab = co_occurrence_matrix(tokens)

## Consult Bigram Frequency

In [ ]:
consultable_matrix = matrix.tocsr()

In [ ]:
def consult_frequency(w1, w2):
    return(consultable_matrix[vocab[w1],vocab[w2]])

# Código do lab 1 para pesquisa OR.

In [ ]:
def preencher( tupla ):
    palavra, id_texto = tupla
    if palavra.lower() not in dados_TF[id_texto]:
        dados_TF[id_texto][palavra.lower()] = 0
    dados_TF[id_texto][palavra.lower()] += 1
    try:
        indice_invertido [ palavra.lower() ].add(id_texto)
    except KeyError:
        indice_invertido[ palavra.lower() ] = set([ id_texto ])

In [ ]:

def extrair( texto , id_texto ):
    for i in xrange( len(texto) ):
        dados_TF[id_texto[i]]= {}
        for palavra in nltk.word_tokenize( str(texto[i]).decode('utf-8') ):
            preencher((palavra,id_texto[i]))



In [ ]:
if __name__ == "__main__":
    file_csv = pd.read_csv("../data/estadao_noticias_eleicao.csv")
    file_csv = file_csv.replace(np.nan, '', regex=True)
    texto = file_csv.titulo + " " + file_csv.conteudo + " " + file_csv.subTitulo
    extrair(texto,file_csv.idNoticia)

# Código do Lab2 - Expansão de Consultas.

##Escreva uma função que receba um certo termo de consulta e a matriz construída no passo 1 acima e retorne as top-3 palavras em ordem decrescente de frequencia.

A função pega os tokens(todas as palavras do dicionário), faz um set para tirar as repetições, depois tranforma e lista denovo. Então eu crio uma lista(dic), contendo tuplas de (palavra, frequência da palavra com o termo). Após a criação dessa lista, eu ordeno ela pela frequência, e pego as três palavras com maior frequência e retorno elas. 
OBS: Crio essa lista de nome, frequencia, ultilizando a função auxiliar consult_frequency(word, i), criada pelos monitores da turma, então eu não preciso receber na função a matrix de co-ocorrência, porque essa função já usa a matriz.

In [ ]:
def top3_similarity(word):
    words = set(tokens)
    words = list(words)
    dic = []
    for i in words:
        try:
            dic.append((i, consult_frequency(word, i)))
        except KeyError:
            return ["Not Found: palavra '"+word+"' não encontrada"]
    dic.sort(key=lambda tup: tup[1], reverse=True)
    similar_words = []
    for tupla in dic[:3]:
       similar_words.append(tupla[0])
    return similar_words

##Expanda a consulta original com os termos retornados no passo 2 acima.

Essa função recebe o termo a ser procurado e retorna a lista expandida com os top3 palavras mais similares.

In [ ]:
def expand_consult(word):
    words = []
    words.append(word)
    words += top3_similarity(word)
    return words

##Faça uma busca disjuntiva (OR) considerando a nova consulta.

**Função de busca OR Binário.**
Função de busca que retorna os documentos que possuem pelo menos uma das palavras da consulta. Função de labs anteriores é uma função de busca binária.

In [ ]:
def procurarOR(lista):
    indices = indice_invertido[lista[0].lower()]
    for palavra in lista:
        indices = set(reduce( lambda x,y : list( x | y ), [ indice_invertido[palavra.lower()] , indices] ))
    return indices

##Quais os termos retornados para a expansão de cada consulta?

In [ ]:
term1 = "poucos"
term2 = "recursos"
term3 = "corinthians"

terms1 = top3_similarity(term1)

terms2 = top3_similarity(term2)

terms3 = top3_similarity(term3)


##Consulta **poucos**:

In [ ]:
print("Termos encontrados na expansão da pesquisa de "+term1+":")
for i, word in enumerate(terms1):
    print i+1,word

**Você acha que esses termos são de fato relacionados com a consulta original? Justifique.**

Sim, pois são termos que em conjunto fazem sentido. Exemplo: poucos dias, poucos meses, poucos metros.

##Consulta **recursos**:

In [ ]:
print("Termos encontrados na expansão da pesquisa de "+term2+":")
for i, word in enumerate(terms2):
    print i+1,word

**Você acha que esses termos são de fato relacionados com a consulta original? Justifique.**

Sim, mas acho que só duas das palavras fazem sentido, que são públicos e petrobrás. Recursos fundo não parece fazer muito sentido em conjunto, mas pode ser porque a frase não está completa, poderia ter outras palavras após essas que na frase fariam mais sentido. Já, recursos públicos e recursos petrobrás parece fazer sentido, deve possivelmente está relacionado ao interesse dos usuários em saber como estão sendo gastos os recursos do país.

##Consulta **corinthians**:

In [ ]:
print("Termos encontrados na expansão da pesquisa de "+term3+":")
for i, word in enumerate(terms3):
    print i+1,word

**Você acha que esses termos são de fato relacionados com a consulta original? Justifique.**

Sim, pois andrés é o presidente do timão. 2010 deve ter sido a pesquisa em alta no ano da captação desses dados, procurando saber como o corinthians foi ou que jogos ele teria no ano de 2010. Paulo pode ser uma palavra ambígua, tanto pode representa o estado de São paulo, ou algum jogador chamado paulo que fez sucesso no corinthians na época, confesso que não lembro de nenhum jogador com esse nome.

**Compare os documentos retornados para a consulta original com a consulta expandida. Quais resultados você acha que melhor capturam a necessidade de informação do usuário? Por que?**

In [ ]:
print(procurarOR([term1]))
print(procurarOR(terms1))

#A expansão de consultas é mais adequada para melhorar o recall ou o precision? Por que?